[View in Colaboratory](https://colab.research.google.com/github/tcglarry/aoi/blob/master/aoi_densenet_save_model_output.ipynb)

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.preprocessing import image

from keras.layers import Dense, GlobalAveragePooling2D
import time

import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback

Using TensorFlow backend.


In [0]:
#src = 'drive/aoi_test/'
src= ''

In [0]:
train_img = os.listdir(src+'train_images')
print ('iamge lies size', len(train_img))
img_list=[]
for i,img in enumerate(train_img):
    x = cv2.imread(src+'train_images/'+img)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    #x = x/127.5 -1.
    img_list.append(x)
    
print ('done')

iamge lies size 2528
done


In [0]:
train_data = np.array(img_list)
print (train_data.shape)
#train_data =  train_data[:,:,:,np.newaxis]
print (train_data.shape)

(2528, 512, 512, 3)
(2528, 512, 512, 3)


In [0]:
print ('here')

here


In [0]:
inputs = Input(shape=(512,512,3))
inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
# create the base pre-trained model
base_model = DenseNet121(weights='imagenet',input_tensor=inputs, input_shape=(512,512,3), include_top=False)
x = base_model.output
print (x.get_shape)
model = Model(inputs=base_model.input, outputs=x)
model.summary()

<bound method Tensor.get_shape of <tf.Tensor 'bn/cond/Merge:0' shape=(?, 16, 16, 1024) dtype=float32>>
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
______

In [0]:
s = time.time()
prediction = model.predict(train_data)
np.save('save_test.npy')
e = time.time()
print ('model saved, total time=', round(e - s, 4))

In [0]:
print (prediction.shape)

NameError: name 'prediction' is not defined